In [10]:
# documentation scipy.stats.norm: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html

In [20]:
from scipy.stats import norm
import numpy as np

In [64]:
# EMSR a with 3 classes
r = [300, 200, 100, 50, 25]  # prices
mu = [10, 15, 25, 50, 500]  # means
sigma = [2, 2, 3, 3, 5]  # sds
C = 160  # capacity

range(0, 5)


In [65]:
def calculate_emrs_a(prices, means, sds, capacity):
    n = len(prices)
    G = []
    
    for i in range(n):
        G_i = 0
        for j in range(i + 1):
            if j == 0:
                ppf_value = 1 - (prices[i] / prices[0])
                if ppf_value < 0:
                    G_j = 0
                else:
                    G_j = np.round(norm(means[j], sds[j]).ppf(ppf_value), 0)
            else:
                ppf_value = 1 - (prices[i] / prices[j - 1])
                if ppf_value < 0:
                    G_j = 0
                else:
                    G_j = np.round(norm(means[j], sds[j]).ppf(ppf_value), 0)

            G_i += G_j

        G.append(G_i)

    G.append(capacity)

    return G

# Beispiel für die Verwendung der Funktion
prices = [300, 200, 100, 50, 25]  # Preise
means = [10, 15, 25, 50, 500]  # Mittelwerte
sds = [2, 2, 3, 3, 5]  # Standardabweichungen
capacity = 160  # Kapazität

results = calculate_emrs_a(prices, means, sds, capacity)
for index, value in enumerate(results):
    print(f"G_{index + 1}: {value}")


G_1: -inf
G_2: 23.0
G_3: 52.0
G_4: 106.0
G_5: 611.0
G_6: 160


In [54]:
G_2_1 = np.round(norm(mu[0], sigma[0]).ppf(1-r[1]/r[0]),0)
G_1 = G_2_1
print(G_1)

G_3_1 = np.round(norm(mu[0], sigma[0]).ppf(1-r[2]/r[0]),0)
G_3_2 = np.round(norm(mu[1], sigma[1]).ppf(1-r[2]/r[1]),0)
G2 = G_3_1 + G_3_2
print(G2)

G_4_1 = np.round(norm(mu[0], sigma[0]).ppf(1-r[3]/r[0]),0)
G_4_2 = np.round(norm(mu[1], sigma[1]).ppf(1-r[3]/r[1]),0)
G_4_3 = np.round(norm(mu[2], sigma[2]).ppf(1-r[3]/r[2]),0)
G3 = G_4_1 + G_4_2 + G_4_3
print(G3)

G_5_1 = np.round(norm(mu[0], sigma[0]).ppf(1-r[4]/r[0]),0)
G_5_2 = np.round(norm(mu[1], sigma[1]).ppf(1-r[4]/r[1]),0)
G_5_3 = np.round(norm(mu[2], sigma[2]).ppf(1-r[4]/r[2]),0)
G_5_4 = np.round(norm(mu[3], sigma[3]).ppf(1-r[4]/r[3]),0)
G4 = G_5_1 + G_5_2 + G_5_3 + G_5_4
print(G4)

G_5 = C
print(G_5)



# Values for G2
G_3_1 = np.round(norm(mu[0], sigma[0]).ppf(1-r[2]/r[0]),0)
G_3_2 = np.round(norm(mu[1], sigma[1]).ppf(1-r[2]/r[1]),0)
G2 = G_3_1 + G_3_2
B3 = C - G2

G1 = np.round(norm(mu[0], sigma[0]).ppf(1-r[1]/r[0]),0)  #ppf=percent point function (inverse of cumulative distribution function)
B2 = C-G1
B1 = C
G3 = C


9.0
26.0
53.0
107.0
160


In [4]:
print("G1: {}".format(G1))
print("G2: {}".format(G2))
print("G3: {}".format(G3))
print("B1: {}".format(B1))
print("B2: {}".format(B2))
print("B3: {}".format(B3))


G1: 36.0
G2: 62.0
G3: 160
B1: 160
B2: 124.0
B3: 98.0


In [15]:
# EMSR b with 3 classes
r = [300, 100, 50]  # prices
mu = [35, 25, 500]  # mean values
sigma = [2, 2, 2]  # SDs
C = 160  # capacity

In [16]:
# G2
mu_2_est = mu[0] + mu[1] # estimated mu
sigma_2_est = np.sqrt(sigma[0]**2 + sigma[1]**2)  # estimated sigma
r_2_est = (mu[0]*r[0] + mu[1]*r[1])/(mu[0]+mu[1]) # weighted mean   
G2 = np.round(norm(mu_2_est, sigma_2_est).ppf(1-r[2]/r_2_est),0)
B3 = C-G2
G1 = np.round(norm(mu[0], sigma[0]).ppf(1-r[1]/r[0]),0)  #ppf=percent point function (inverse of cumulative distribution function)
B2 = C-G1
B1= C
G3 = C

In [17]:
print("G1: {}".format(G1))
print("G2: {}".format(G2))
print("G3: {}".format(G3))
print("B1: {}".format(B1))
print("B2: {}".format(B2))
print("B3: {}".format(B3))

G1: 36.0
G2: 62.0
G3: 160
B1: 160
B2: 124.0
B3: 98.0
